In [ ]:
from litellm import completion
import re
from typing import Dict, List
import json
import csv


In [ ]:
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
     response = completion(
        model="gemini/gemini-2.0-flash",
        messages=messages,
        tools=tools,
        max_tokens=512
    )
    return response.choices[0].message.content

In [ ]:
def gen_events(new_tweets: List[str],events) -> Dict:
    system_prompt = f"""
    You are a sports event{event} summarizer for football (soccer) matches.
    You analyze live tweets and output structured JSON with only *unique, confirmed match events*.

    Rules:
    - Detect and record only confirmed events as well as the player in question:
      goal, disallowed goal (after VAR/offside confirmation), yellow card, red card, substitution, injury, shot on goal, save, half-time, full-time.
    - Ignore non-events such as: VAR checking (before decision), near misses, fan reactions, speculation, or commentary without an actual event.
    - If a tweet includes the event time (e.g., "45+2’", "70th min"), extract it and include it.
    - If the match ends (Full Time, FT, End of Match, final whistle), record it as the last event (only once).
    - Be concise (one short sentence per event).
    - Always return valid JSON only in this format:

    Return only valid JSON with double quotes, no explanations.
Format:
      {{"event": {{"time": "", "event": "text of the event"}},
         ...
         "event": {{"time": "", "event": "text of the event"}}

}}


    """

    user_prompt = f"""

    New tweets:
    {new_tweets}

    generate the events list with any new confirmed events only.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response=generate_response(messages)

    return response

In [ ]:
def get_sum(events):
  system_prompt = """
You are a football match summarizer and stats updater.

Input:
- A list of confirmed match events .
- The current stats dictionary.

Tasks:
1. Write a short neutral summary of the events .
2. Update the stats dictionary by incrementing only the values that match the new events.

Rules:
- Track these stats:
  yellow_cards, offsides, blocked_shots, shots_on_goal, shots_off_goal, goalkeeper_saves.
- Goals are mentioned in the summary but not counted in these stats.
- Only increment stats for new events. Do not reset or decrease values.
- If no events happened, summary should be: "No major events in this period."
- Always return JSON with two fields:
  {
    "summary": "...",
    "stats": { ... }
  }
    """

  user_prompt = f"""
    Events in the last 5 minutes:
Events:
{events}


Generate the summary and updated stats.
    """

  messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

  response=generate_response(messages)
  print(response)
  return response


In [ ]:
def read_in_batches(file_path: str, batch_size: int = 50):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]  # clean & skip empty lines

    # Split into batches of N lines
    for i in range(0, len(lines), batch_size):
        yield lines[i:i+batch_size]


events = {}
min_events = {}

stats = {
  "yellow_cards": 0,
  "offsides": 0,
  "blocked_shots": 0,
  "shots_on_goal": 0,
  "shots_off_goal": 0,
  "goalkeeper_saves": 0
}

In [ ]:

file_path = "New-Text-Document.txt"

for idx, batch in enumerate(read_in_batches(file_path, batch_size=50), start=1):
    print(events)
    inst_event = gen_events(batch)
    inst_eventw = re.sub(r"```json|```|'''", '', inst_event).strip()
    inst_event = json.loads(inst_eventw)
    print('inst_event', inst_event)
    if inst_event != {}:
      update_events(inst_event)
      print('events', events)


